In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "itxgbm" #TODO
fs_method, fs_ratio = "dnp", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [06:46<00:00,  5.73s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 1/50:  42%|████▏     | 11/26 [00:00<00:00, 106.71batch/s, loss=0.289]

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9121 | AUPR: 0.6303 | Acc: 95.15%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9184 | AUPR: 0.6456 | Acc: 95.23%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9218 | AUPR: 0.6590 | Acc: 95.25%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9132 | AUPR: 0.6327 | Acc: 95.13%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9186 | AUPR: 0.6467 | Acc: 95.28%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9210 | AUPR: 0.6556 | Acc: 95.31%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9218 | AUPR: 0.6590 | Acc: 95.25%
Internal test: AUC: 0.9179 | AUPR: 0.6717 | Acc: 94.84%
External test: AUC: 0.9167 | AUPR: 0.6480 | Acc: 94.95%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC


model(kernel=linear, C=0.01)
Valid:         AUC: 0.9158 | AUPR: 0.6155 | Acc: 94.93%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9198 | AUPR: 0.6287 | Acc: 94.99%
model(kernel=linear, C=1)
Valid:         AUC: 0.9195 | AUPR: 0.6292 | Acc: 94.99%
best model(kernel=linear, C=0.1)
Valid:         AUC: 0.9198 | AUPR: 0.6287 | Acc: 94.99%
Internal test: AUC: 0.9154 | AUPR: 0.6539 | Acc: 94.65%
External test: AUC: 0.9127 | AUPR: 0.6089 | Acc: 94.63%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7572 | AUPR: 0.4055 | Acc: 94.37%
model(n_neighbors=5)
Valid:         AUC: 0.7862 | AUPR: 0.4555 | Acc: 94.80%
model(n_neighbors=7)
Valid:         AUC: 0.8039 | AUPR: 0.4785 | Acc: 94.87%
best model(n_neighbors=7)
Valid:         AUC: 0.8039 | AUPR: 0.4785 | Acc: 94.87%
Internal test: AUC: 0.7988 | AUPR: 0.4965 | Acc: 94.65%
External test: AUC: 0.8044 | AUPR: 0.4824 | Acc: 94.72%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8670 | AUPR: 0.4277 | Acc: 82.60%
best model()
Valid:         AUC: 0.8670 | AUPR: 0.4277 | Acc: 82.60%
Internal test: AUC: 0.8603 | AUPR: 0.4333 | Acc: 82.04%
External test: AUC: 0.8544 | AUPR: 0.3803 | Acc: 80.97%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9212 | AUPR: 0.6481 | Acc: 95.26%
model(C=10.0)
Valid:         AUC: 0.9212 | AUPR: 0.6478 | Acc: 95.23%
model(C=100.0)
Valid:         AUC: 0.9207 | AUPR: 0.6475 | Acc: 95.22%
best model(C=1.0)
Valid:         AUC: 0.9212 | AUPR: 0.6481 | Acc: 95.26%
Internal test: AUC: 0.9176 | AUPR: 0.6635 | Acc: 94.79%
External test: AUC: 0.9118 | AUPR: 0.6214 | Acc: 94.76%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8674 | AUPR: 0.5463 | Acc: 82.33%
model(criterion=log_loss)
Valid:         AUC: 0.8707 | AUPR: 0.5697 | Acc: 82.88%
model(criterion=entropy)
Valid:         AUC: 0.8707 | AUPR: 0.5697 | Acc: 82.88%
best model(criterion=log_loss)
Valid:         AUC: 0.8707 | AUPR: 0.5697 | Acc: 82.88%
Internal test: AUC: 0.8653 | AUPR: 0.5989 | Acc: 85.89%
External test: AUC: 0.8728 | AUPR: 0.5885 | Acc: 85.82%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9124 | AUPR: 0.6408 | Acc: 88.08%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9129 | AUPR: 0.6431 | Acc: 88.14%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9133 | AUPR: 0.6433 | Acc: 88.28%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9107 | AUPR: 0.6382 | Acc: 88.22%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9123 | AUPR: 0.6425 | Acc: 88.16%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9126 | AUPR: 0.6438 | Acc: 88.38%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9107 | AUPR: 0.6382 | Acc: 88.22%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9123 | AUPR: 0.6425 | Acc: 88.16%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9126 | AUPR: 0.6438 | Acc: 88.38%
best model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9133 | AUPR: 0.6433 | Acc: 8

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9218 | AUPR: 0.6544 | Acc: 95.32%
model(criterion=squared_error)
Valid:         AUC: 0.9217 | AUPR: 0.6557 | Acc: 95.33%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9218 | AUPR: 0.6544 | Acc: 95.32%
Internal test: AUC: 0.9258 | AUPR: 0.6960 | Acc: 95.26%
External test: AUC: 0.9232 | AUPR: 0.6645 | Acc: 95.23%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier
model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9146 | AUPR: 0.6238 | Acc: 95.10%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9155 | AUPR: 0.6243 | Acc: 95.07%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9163 | AUPR: 0.6267 | Acc: 95.04%
best model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9163 | AUPR: 0.6267 | Acc: 95.04%
Internal test: AUC: 0.9207 | AUPR: 0.6699 | Acc: 95.16%
External test: AUC: 0.9162 | AUPR: 0.6169 | Acc: 94.74%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)